In [1]:
%matplotlib inline

TRAIN_DATA = "../mscoco_subset_cs601r/train/"
TRAIN_DATA_LABEL = "../mscoco_subset_cs601r/train_masks/"
TEST_DATA = "../mscoco_subset_cs601r/test/"
TEST_DATA_LABEL = "../mscoco_subset_cs601r/test_masks/"

In [2]:
import torch.nn as nn
import torchvision.models as models
from Data import Data
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import time
import pickle
import numpy as np
from SegNet import SegNet

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
seg = SegNet()
seg.to(device)
try:
    seg.load_state_dict(torch.load('model.json'))
except:
    pass
dataset = Data(TRAIN_DATA, TRAIN_DATA_LABEL)
loader = DataLoader(dataset, batch_size=22, shuffle=True, num_workers=10)

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(seg.parameters(), lr=5e-4, momentum=0.9)
optimizer = optim.Adam(seg.parameters(), lr=5e-4)
try:
    with open('running_loss.pkl', 'rb') as f:
        running_loss = pickle.load(f)
except:
    running_loss = []

very_start = time.time()
for epoch in range(1, 4):
    start = time.time()
    epoch_loss = []
    for i, data in enumerate(loader, 0):
        images, labels = data[0].to(device), data[1].to(device)
        images = images.type(torch.FloatTensor).cuda()
        labels = labels.type(torch.LongTensor).cuda()
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = seg(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        loss_val = loss.item()
        # print(i, ":", loss_val)
        epoch_loss.append(loss_val)

    print('[Epoch:', epoch, '] [Time:',
          (time.time()-start)/60, " minutes] [Average loss:", np.average(epoch_loss), "]")
    running_loss += epoch_loss
    torch.save(seg.state_dict(), 'model.json')
    with open('running_loss.pkl', 'wb') as f:
        pickle.dump(running_loss, f)
print('Finished Training in ', (time.time()-very_start)/60, " minutes")
print("length of loss array: ", len(running_loss))